In [4]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [5]:
with open('database/intents.json') as file:
    data = json.load(file)

In [6]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [7]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [8]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [9]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 10)                170       
                                                                 
Total params: 16,714
Trainable params: 16,714
Non-trainable params: 0
____________________________________________________

In [11]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
2/2 [==============================] - 1s 4ms/step - loss: 2.3045 - accuracy: 0.0455
Epoch 2/550
2/2 [==============================] - 0s 2ms/step - loss: 2.3030 - accuracy: 0.1136
Epoch 3/550
2/2 [==============================] - 0s 3ms/step - loss: 2.3020 - accuracy: 0.1136
Epoch 4/550
2/2 [==============================] - 0s 3ms/step - loss: 2.3011 - accuracy: 0.1136
Epoch 5/550
2/2 [==============================] - 0s 3ms/step - loss: 2.3002 - accuracy: 0.1818
Epoch 6/550
2/2 [==============================] - 0s 4ms/step - loss: 2.2994 - accuracy: 0.1818
Epoch 7/550
2/2 [==============================] - 0s 2ms/step - loss: 2.2989 - accuracy: 0.1364
Epoch 8/550
2/2 [==============================] - 0s 3ms/step - loss: 2.2983 - accuracy: 0.1136
Epoch 9/550
2/2 [==============================] - 0s 3ms/step - loss: 2.2973 - accuracy: 0.1136
Epoch 10/550
2/2 [==============================] - 0s 4ms/step - loss: 2.2965 - accuracy: 0.1136
Epoch 11/550
2/2 [===========

In [12]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets
